In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType

In [2]:
# ====================== INITIALISATION ======================
spark = SparkSession.builder \
    .appName("TrainALSModel") \
    .master("yarn") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/02 10:17:17 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
# ====================== CHARGEMENT DES DONNÉES ======================
ratings_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", FloatType(), True),
    StructField("timestamp", StringType(), True),
])

movies_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("genres", StringType(), True),
])

print("📥 Lecture des fichiers CSV depuis HDFS...")
ratings_df = spark.read.csv("hdfs:///input/rating.csv", header=True, schema=ratings_schema)
movies_df = spark.read.csv("hdfs:///input/movie.csv", header=True, schema=movies_schema)

📥 Lecture des fichiers CSV depuis HDFS...


In [4]:
# ====================== NETTOYAGE ======================
# Supprimer les lignes avec valeurs nulles
ratings_df = ratings_df.dropna(subset=["userId", "movieId", "rating"])

In [5]:
# ====================== DIVISION TRAIN / TEST ======================
train_df, test_df = ratings_df.randomSplit([0.8, 0.2], seed=42)

In [6]:
# ====================== ENTRAÎNEMENT ======================
print("🤖 Entraînement du modèle ALS...")
als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    nonnegative=True,
    implicitPrefs=False,
    coldStartStrategy="drop",  # pour éviter les NaN en test
    rank=13,
    maxIter=19,
    regParam=0.18
)

model = als.fit(train_df)

🤖 Entraînement du modèle ALS...


In [7]:
# ====================== ÉVALUATION ======================
print("📊 Évaluation du modèle...")
predictions = model.transform(test_df)

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)
print(f"✅ RMSE sur l'ensemble test : {rmse:.4f}")

📊 Évaluation du modèle...


[Stage 184:============================>                            (2 + 2) / 4]

✅ RMSE sur l'ensemble test : 0.8522


In [8]:
# ====================== ENREGISTREMENT ======================
print("💾 Sauvegarde du modèle dans HDFS (/models/als)...")
model.write().overwrite().save("hdfs:///models/als")

print("🎉 Modèle entraîné et sauvegardé avec succès.")

💾 Sauvegarde du modèle dans HDFS (/models/als)...


[Stage 268:==================================================>     (9 + 1) / 10]

🎉 Modèle entraîné et sauvegardé avec succès.


In [9]:
spark.stop()